In [1]:
!pip install -r requirements.txt

In [2]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
import qdrant_client
import os

In [3]:
client = qdrant_client.QdrantClient(
    "localhost", port=6333
)

In [4]:
vectors_config = qdrant_client.http.models.VectorParams(
    size=384,
    distance=qdrant_client.http.models.Distance.COSINE
)

In [5]:
client.recreate_collection(
    collection_name="IETFDocs",
    vectors_config=vectors_config,
)

True

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/home/moyankgiri/anaconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def load_documents():
    loader = DirectoryLoader('/home/moyankgiri/Desktop/LLM/docs/', glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(text)
    return chunks

In [9]:
documents = load_documents()
text_chunks = get_chunks(documents)

In [10]:
qdrant = Qdrant.from_documents(
    text_chunks,
    embeddings,
    location="localhost:6333",
    prefer_grpc=True,
    collection_name='IETFDocs',
    force_recreate=True
)

In [11]:
from llama_index.core.prompts.prompts import SimpleInputPrompt
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [12]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved to /home/moyankgiri/.cache/huggingface/token
Login succe

In [13]:
import torch
torch.set_default_device('cuda')

In [14]:
# from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain import HuggingFacePipeline
# llm = HuggingFaceLLM(
#     generate_kwargs={"temperature": 0.1, "do_sample": True},
#     system_prompt=system_prompt,
#     query_wrapper_prompt=query_wrapper_prompt,
#     tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
#     model_name="mistralai/Mistral-7B-Instruct-v0.1",
#     device_map="auto",
#     model_kwargs={"torch_dtype": torch.float16}
# )

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"torch_dtype": torch.float16},
) 

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.89s/it]


In [15]:
def get_vectore_store():
    
    client = qdrant_client.QdrantClient(
        "localhost", port=6333
    )
    
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    vectore_store = Qdrant(
        client=client,
        collection_name="IETFDocs",
        embeddings=embeddings
    )
    
    return vectore_store

In [16]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)        

In [17]:
from langchain.chains import RetrievalQA
vectore_store = get_vectore_store()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=vectore_store.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

In [18]:
retriever = qdrant.as_retriever()
retriever.get_relevant_documents("What can the conversion to PDF generate?")

[Document(page_content='RFC 7995                      PDF for RFCs                 December 2016\nC.3.  PDF Generation Libraries\n   Because the xml2rfc format is a unique format, software for\n   converting XML source documents to the various formats will be\n   needed, including PDF generation.\n   One promising direction is suggested in\n   <http://greenbytes.de/tech/webdav/rfc2629xslt/\n   rfc2629xslt.html#output.pdf.fop>: using XSLT (Extensible Stylesheet\n   Language Transformations) to generate XSL-FO (XSL Formatting\n   Objects); XSL-FO is then processed by a FOP (Formatting Objects\n   Processor) such as Apache FOP.\n   Several libraries are also available for generating PDF signatures.\n   The choice of library to use for xml2pdf will depend on many factors:\n   programming language, quality of implementation, quality of PDF\n   generated, support, cost, availability, and so forth.\nC.4.  Typefaces\n   Various typefaces are available that might satisfy the requirements', meta

In [19]:
result = qa ({ "query" : "What can the conversion to PDF generate?" },return_only_outputs=True) 
print(result["result"])   

/home/moyankgiri/anaconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
RFC 7995                      PDF for RFCs                 December 2016
C.3.  PDF Generation Libraries
   Because the xml2rfc format is a unique format, software for
   converting XML source documents to the various formats will be
   needed, including PDF generation.
   One promising direction is suggested in
   <http://greenbytes.de/tech/webdav/rfc2629xslt/
   rfc2629xslt.html#output.pdf.fop>: using XSLT (Extensible Stylesheet
   Language Transformations) to generate XSL-FO (XSL Formatting
   Objects); XSL-FO is then processed by a FOP (Formatting Objects
   Processor) such as Apache FOP.
   Several libraries are also available for generating PDF signatures.
   The choice of library to use for xml2pdf will depend on many factors:
   programming language, quality of implementation, qu